# Step1: Supervised Finetuning
Problems occuring: repeating content generation & inconsistency between perplexit(PPL) scores & generation capabilities

PPL scores $p(sentence)^{-1/N}$

serveral terms affect the generation behavior:

+ weight decay: OPT(Open Pre-trained Transformer Language Models) are pretrained with weight decay, althought we should inherits this setting, it may not produce the desired model.
+ dropout: enabled in both OPT and Fintuning training
+ dataset: using more data usually provides better model quality. Yet if the sources of datasets are too different, it may hurt the performance. For our OPT-1.3B example, we use the following four datasets: *Dahoas/rm-static  Dahoas/full-hh-rlhf, Dahoas/synthetic-instruct-gpth-piarwise, yitingxie/rlhf-reward-datasets*
+ training epochs: Normally, to avoid overfitting, we hcoose smaller training epochs. However similar to InstructGPT pointed, we found even though we got overfitting dut to longer training, it is still recommended to use longer training epochs to get better generation quality. Particularly for our OPT-1.3B model, we use 16 epochs even though 1 ro 2 epochs training can reach the same PPL score.



# Step2: Reward Model Finetuning

difference from SFT

+ training datasets: RM requires both good response and bad responses.
+ training loss: RM requires pair ranking loss as the optimizing objective.
  
metrics for the reward model

+ the reward score for accepted response(and bad reponses),
+ the accuracy, i.e., when accecpted responses can get higher scores.

no definitive answer about reward score gain/increasing and negative average reward score for accepted answers

Observation during our exploration(OPT-350M):
+ weight decay: enabled
+ dropout: disabled
+ dataset: same four datasets
+ training: InstructGPT suggests to finetune the model with 1 epoch since overfitting hurts the RLHF performance. During our expolration. wedid not see overfitting behavior when we increased the training epochs. However, to follow the instructions from the authors, we set traning epoch to be 1.

Other explorations:
+ multiple answers for one prompt: Instruct GPT constructs the dataset with 4--9 answers per prompt.
+ initialize RM with SFT or Pretrained checkpoint: no big difference for either accuracy or reward score.
+ Reward score calculation: we use the final token(or the first padding token) to get the reward score, not sure if it is  optimal.
+ Reward loss objective: ranking loss was adopted, yet others, like MSE, can also be an option.
 


# Step 3: RLHF fintuning

Similar to SFT, the reward score cannot really reflect the model generation quality. Also, we sometimes observed that the reward score drops to the inital phase at a certain point and then quickly recovers. To make things worse, we also see the training can easily get divergence.

Our settings and observations:
+ weight decay: disabled for both actor and critic models
+ dropout: disabled droppout for OPT-1.3B and enabled it for OPT-350M
+ dataset: single dataset: *Dahoas/rm-static*
+ training epochs: The reward score quickly becomes platou. Therefore, we set the traning epoch to be 1. Yet like SFT, longer training may bring better model quality as SFT.
+ ema checkpoing: as stated in InstructGPT, ema checkpoint can generally bring better model generation quality.
+ PPO related hyperparameters: 
+ mix unsupervised training: InstructGPT suggests mixing PPO and unsupervised training to prevent the loss of the model's benchmark quality.
+ diverging issue: We have found that it is very unstable to use different generation training batch sizes(--per_device_generation_batch_size) and PPO training batch sizes(--per_device_training_batch_size), more than one PPO training epoch(--ppo_epochs), or more than one generation batch(--generation_batches 1). These all point to the same problem: we are not able to update the actor model multiple times after generating experimental data. Therefore, in all of our successful runs, we have set
$per\_device\_generation\_batch\_size = per\_device\_training\_batch\_size$ and
$ppo\_epochs=generation\_batches = 1$.
This is unexpected for a standard RL training pipeline, and we have tried different methods to overcome this, but all have failed. Obe of the most likely reasons for this instability is that we found the log_probs and old_log_probs used in the actor_loss_fn function can quickly diverge even within two consecutive iterations, which causes the corresponding ratio to be huge. Setting a strict upper bound can alleviate this problem, but it cannot fully resolve the convergence issue.